In [36]:
%serialconnect to --port=/dev/ttyUSB1 --baud=115200

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [37]:
from m5stack import *

lcd.clear(lcd.BLACK)
lcd.text(lcd.CENTER, lcd.CENTER, 'my name is franek_b')


In [ ]:
  Serial2.begin(19200, SERIAL_8N1, 0, 26);         // RX=0   TX=26
    Serial2.print("@OF30");           // Offset
    Serial2.write(0x0a);
    Serial2.print("@RG2");            // MAG=2
    Serial2.write(0x0a);
    fConv = (float)LCDHEIGHT/(float)MAXADRANGE;  

In [68]:
# Set uart port and tx rx
uart = machine.UART(1)

# Set part baud and data bits, Parity bits, stop bits
#uart.init(19200, tx= 0,rx = 26)#, bits, parity, stop)

uart.init(19200, tx = 26, rx = 0)
# Read all data from uart
#uart.read()

# Read 10 characters from uart
#uart.read(10)

# Read a line data from uart
#uart.readline()

In [69]:
uart_out_buf = bytearray(1)
uart.write("@OF30")
uart_out_buf = 0x0a
uart.write(b'\x0a')
uart.write("@RG2")
uart_out_buf = 0x0a
uart.write(b'\x0a')

In [70]:
uart.write("jak tam na swiecie \nwszystko ok?\n")

In [83]:
while True:
    result = uart.readline()
    
    if result:
        number = result.decode()
        
        if number[0] == "#": #special case
            if number[1] == "-":
                pass
                #print("error")
            else:
                heartrate = 60000.0 / float(number[1:])
                
                lcd.clear(lcd.BLACK)

                lcd.text(lcd.CENTER, lcd.CENTER, str(heartrate))

                #print("heart rate",heartrate)
        else:
        #
            print(number.strip())

1059
1051
1091
1126
1149
1159
1220
1273
1362
1400
1502

2425
2358
2340
2302
2269
2209
2185
2156
2120
2068
2050
2014
2004
1958
1937
1921
1906
1862
1839
1827
1809
1769
1754
1747
1742
1722
1691
1694
1686
1669
1635
1641
1620
1607
1567
1567
1541
1539
1500
1504
1484
1485
1443
1466
1451
1459
1434
1448
1460
1466
1453
1470
1494
1508
1499
1514
1545
1557
1559
1560
1585
1595
1592
1586
1615
1606
1604
1572
1580
1575
1561200
1233
1180
1143
1182
1210
1158
1129
1170
1203
1157
1096
1137
1193
1182
1106
1148
1201
1181
1107
1139
1198
1201
1134
1140
1167
1196
1124
1140
1168
1196
1124
1124
1169
1208
1162
1142
1188
1228
1178
1157
1199
1244
1199
1160
1186
1206
1188
1142
1169
1185
1165
1118
1143
1168
1155
1073
1087
1123
1112
1025
1036
1085
1078
996
994
1047
1073
1005
991
1040
1074
1006
982
1029
1066
1043
979
1035
1081
1055
986
1030
1086
1059
996
1030
1050
1056
983
1009
1028
1049
976
999
1028
1042
956
950
1000
1023
953
948
992
1025
959
947
997
1043
1009
972
1014
1046
1011
966
1001
1029
996
909
940
986
972
886
91

Traceback (most recent call last):
  File "<stdin>", line 8, in <module>
KeyboardInterrupt: 


In [ ]:
/*----------------------------------------------
 *  M5StickC Plus Pulse Serial Recieve Sample
 *    #2019/06/06 - #2019/06/08 - #2020/07/22
 *    Serial 19200ボー
 *    No Speaker
 *    
 *    Board Select -> M5Stick-C
  -----------------------------------------------*/
#include <M5StickCPlus.h>
#include <WiFi.h>

#define MAXADRANGE  4096
#define DELIMITCODE  0x0a       // Delimit Code 

#define SWEEPMAG    8 
#define LCDWIDTH    240      // M5Stick-Plus  240
#define LCDHEIGHT   135      // M5Stick-Plus  135
#define PLSPOSX     32
#define PLSPOSY     2

static  int sweepptr = 0;
static  int lastY = 0;
static  int plsrate = 0;
static  int x = 0;
static  float fConv;  

void setup() {
    M5.begin();
    pinMode(36, INPUT);
    gpio_pulldown_dis(GPIO_NUM_25);
    gpio_pullup_dis(GPIO_NUM_25);    
//---
    M5.Axp.ScreenBreath(8);         // LCD輝度を抑える 7-15
    M5.Lcd.setRotation(1);          // 回転をDefaultに戻す
    M5.Lcd.fillScreen(BLACK);
//--- 省電力でWifiOFFする場合に下記入れる
    WiFi.mode(WIFI_OFF);
//--
    M5.Lcd.setTextColor(GREEN);
    M5.Lcd.setTextSize(3);    
//--
    delay(500);
    Serial2.begin(19200, SERIAL_8N1, 0, 26);         // RX=0   TX=26
    Serial2.print("@OF30");           // Offset
    Serial2.write(0x0a);
    Serial2.print("@RG2");            // MAG=2
    Serial2.write(0x0a);
    fConv = (float)LCDHEIGHT/(float)MAXADRANGE;  
    WiFi.mode(WIFI_OFF);
}

void loop() {
String   strInput;  
int      val,y;

 if(Serial2.available() > 0) {
    strInput = Serial2.readStringUntil(DELIMITCODE);
    if (strInput[0]=='#'){    
   // 脈拍数データ識別
      strInput[0]=' ';        // @をSpaceに置き換える
      if (strInput[1]=='-'){
        strInput[1]=' ';          // マイナスはErrorマークなのでSpaceに置き換えて色を赤にする
        M5.Lcd.setTextColor(RED);     // エラー時は赤文字
      }else{
        M5.Lcd.setTextColor(GREEN);   // 正常脈拍時は緑文字
      }
      val = strInput.toInt();     
      if (val>0) plsrate = 60000/val;                     // Interval(ms)を1分間の脈拍数(bpm)に変換
      M5.Lcd.fillRect(PLSPOSX, PLSPOSY, 50, 16, BLACK);   //　脈拍数表示領域だけ背景を抜く
    }else{                
  // 脈波形データ処理
      sweepptr++;
      if (sweepptr>=SWEEPMAG){
        sweepptr = 0;
        val = strInput.toInt();       
        M5.Lcd.drawLine(x, 0, x, LCDHEIGHT-1, BLACK);
        y = LCDHEIGHT - (int)((float)val*fConv);
        if (x > 0) {
          M5.Lcd.drawLine(x - 1, lastY, x, y, WHITE);
        }else{
          M5.Lcd.drawLine(x, lastY, x, y, WHITE);
        }
        lastY = y;
        x++;
        if (x >= LCDWIDTH) x=0;
      }
    }
  //　脈拍数を波形の上に上書き    
    M5.Lcd.setCursor(PLSPOSX, PLSPOSY);
    M5.Lcd.println(plsrate);      
  }
}